In [1]:
## TASK 1 -> LOAD DATA AND IMPORT PANDAS

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import mean_absolute_error, confusion_matrix, precision_score, recall_score


In [2]:
## DATA AUDIT
df = pd.read_csv(r"C:\Users\User\Downloads\hospital_outpatient_DataSet.csv")

print("Shape:", df.shape)
print("\nHead:")
print(df.head())

print("\nInfo:")
df.info()

print("\nMissing Values Report:")
print(df.isna().sum().sort_values(ascending=False))

print("\nDuplicate Rows:", df.duplicated().sum())


Shape: (300, 13)

Head:
  Visit_ID        Date   Department Doctor_Name  Patient_Age Gender Symptoms  \
0    V1170  01-01-2024   Cardiology   Dr. Ahmed         61.0      F  Allergy   
1    V1221  01-01-2024   Cardiology   Dr. Ahmed         67.0      M  Fatigue   
2    V1016  01-01-2024   Cardiology   Dr. Meena         48.0      F  Allergy   
3    V1205  03-01-2024  Orthopedics   Dr. Meena         34.0      F      NaN   
4    V1212  03-01-2024  Orthopedics   Dr. Meena         29.0      F    Fever   

      Diagnosis  Medicine_Cost  Consultation_Fee Followup_Required  \
0  Hypertension         1674.0             263.0                No   
1      Migraine          764.0             359.0               Yes   
2      Fracture         1085.0             727.0               Yes   
3  Hypertension         1893.0             323.0                No   
4     Infection         1056.0             357.0               Yes   

   Visit_Duration  Total_Bill  
0              77        1937  
1         

In [3]:
## TASK 2-> DATA CLEANING

## 1. Fix datatype (Date)
print(df['Date'].dtype)
print("Null dates after conversion:", df['Date'].isna().sum())




object
Null dates after conversion: 0


In [4]:
## 2.Missing Values – Numeric (Median)
print("Missing values after numeric imputation:")
print(df[['Medicine_Cost', 'Consultation_Fee']].isna().sum())


Missing values after numeric imputation:
Medicine_Cost       15
Consultation_Fee     7
dtype: int64


In [5]:
## 3. Missing Values – Categorical (Mode)
print("Missing values after categorical imputation:")
print(df[['Diagnosis', 'Symptoms']].isna().sum())


Missing values after categorical imputation:
Diagnosis    15
Symptoms     15
dtype: int64


In [6]:
## 4. Text Cleaning (strip + casing)
print("BEFORE text cleaning:")
print(df[['Department', 'Gender']].head(3))

df['Department'] = df['Department'].str.strip().str.title()
df['Gender'] = df['Gender'].str.strip().str.upper()

print("\nAFTER text cleaning:")
print(df[['Department', 'Gender']].head(3))
print(df['Gender'].unique())


BEFORE text cleaning:
   Department Gender
0  Cardiology      F
1  Cardiology      M
2  Cardiology      F

AFTER text cleaning:
   Department Gender
0  Cardiology      F
1  Cardiology      M
2  Cardiology      F
['F' 'M' nan]


In [7]:
## 5. Remove Duplicates
print("Duplicates before:", df.duplicated().sum())
df = df.drop_duplicates().reset_index(drop=True)
print("Duplicates after:", df.duplicated().sum())


Duplicates before: 0
Duplicates after: 0


In [8]:
## 6. One extra Improvemeny -> Rename Columns
print("Columns after renaming:")
print(df.columns)


Columns after renaming:
Index(['Visit_ID', 'Date', 'Department', 'Doctor_Name', 'Patient_Age',
       'Gender', 'Symptoms', 'Diagnosis', 'Medicine_Cost', 'Consultation_Fee',
       'Followup_Required', 'Visit_Duration', 'Total_Bill'],
      dtype='object')


In [9]:
## After ALL Cleaning
print("Missing values AFTER cleaning:")
print(df.isna().sum())


Missing values AFTER cleaning:
Visit_ID              0
Date                  0
Department            0
Doctor_Name           0
Patient_Age           9
Gender                5
Symptoms             15
Diagnosis            15
Medicine_Cost        15
Consultation_Fee      7
Followup_Required    15
Visit_Duration        0
Total_Bill            0
dtype: int64


In [12]:
print(df.columns)


Index(['Visit_ID', 'Date', 'Department', 'Doctor_Name', 'Patient_Age',
       'Gender', 'Symptoms', 'Diagnosis', 'Medicine_Cost', 'Consultation_Fee',
       'Followup_Required', 'Visit_Duration', 'Total_Bill', 'High_Bill_Flag',
       'Bill_Check'],
      dtype='object')


In [13]:
## TASK 3 -> FEATURE ENGINEERING (3 NEW COLUMNS)
df['High_Bill_Flag'] = np.where(
    df['Total_Bill'] > df['Total_Bill'].median(), 1, 0
)

# Derived numeric column
df['Bill_Check'] = df['Medicine_Cost'] + df['Consultation_Fee']

# Bucket column (USE Patient_Age, not Age)
df['Age_Group'] = pd.cut(
    df['Patient_Age'],
    bins=[0, 18, 35, 60, 100],
    labels=['Child', 'Young Adult', 'Adult', 'Senior']
)

print(df[['High_Bill_Flag', 'Bill_Check', 'Age_Group']].head())

   High_Bill_Flag  Bill_Check    Age_Group
0               1      1937.0       Senior
1               0      1123.0       Senior
2               1      1812.0        Adult
3               1      2216.0  Young Adult
4               0      1413.0  Young Adult


In [14]:
## TASK 4-> INSIGHTS

## Insight 1 – GroupBy + Aggregation

dept_bill = df.groupby('Department').agg({
    'Total_Bill': 'mean'
}).round(2)

print(dept_bill)
print("Conclusion: Some departments have consistently higher average bills.")


             Total_Bill
Department             
Cardiology      1611.46
Dermatology     1535.45
Ent             1606.12
General         1634.48
Orthopedics     1627.64
Conclusion: Some departments have consistently higher average bills.


In [15]:
## Insight 2 –> sort_values
top_visits = df.sort_values(by='Total_Bill', ascending=False)[
    ['Department', 'Doctor_Name', 'Total_Bill']
].head(5)

print(top_visits)
print("Conclusion: A small number of visits contribute to very high billing.")


      Department Doctor_Name  Total_Bill
161          Ent   Dr. Meena        2791
288  Orthopedics   Dr. Meena        2737
177  Dermatology   Dr. Meena        2641
174      General  Dr. Sharma        2619
49   Dermatology   Dr. Ahmed        2617
Conclusion: A small number of visits contribute to very high billing.


In [16]:
## Insight 3 –> value_counts
followup_dist = df['Followup_Required'].value_counts()
print(followup_dist)
print("Conclusion: Majority of patients do not require follow-up visits.")

Followup_Required
No     152
Yes    133
Name: count, dtype: int64
Conclusion: Majority of patients do not require follow-up visits.


In [21]:
## TASK 5 -> REGRESSION – PREDICT TOTAL BILL

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.impute import SimpleImputer

# Feature & target
X = df.drop(columns=[
    'Total_Bill',
    'Date',
    'Visit_ID'
])
y = df['Total_Bill']

# Categorical & numeric columns
categorical_cols = X.select_dtypes(include=['object', 'category']).columns
numeric_cols = X.select_dtypes(exclude=['object', 'category']).columns

# Preprocessing pipelines
numeric_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='median'))
])

categorical_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer([
    ('num', numeric_transformer, numeric_cols),
    ('cat', categorical_transformer, categorical_cols)
])

# Model pipeline
reg_model = Pipeline([
    ('prep', preprocessor),
    ('model', LinearRegression())
])

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Train
reg_model.fit(X_train, y_train)

# Predict
y_pred = reg_model.predict(X_test)

# Evaluate
mae = mean_absolute_error(y_test, y_pred)

print("MAE:", round(mae, 2))
print("MAE means the average absolute difference between actual and predicted bills.")


MAE: 56.98
MAE means the average absolute difference between actual and predicted bills.


In [25]:
## TASK 6 -> CLASSIFICATION – PREDICT YES / NO

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, precision_score, recall_score
from sklearn.impute import SimpleImputer

# DROP ROWS WITH MISSING TARGET
df_cls = df.dropna(subset=['Followup_Required'])

# Feature & target
X_cls = df_cls.drop(columns=[
    'Followup_Required',
    'Date',
    'Visit_ID'
])

y_cls = df_cls['Followup_Required']


# Column types
categorical_cols = X_cls.select_dtypes(include=['object', 'category']).columns
numeric_cols = X_cls.select_dtypes(exclude=['object', 'category']).columns

# Preprocessing
numeric_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='median'))
])

categorical_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor_cls = ColumnTransformer([
    ('num', numeric_transformer, numeric_cols),
    ('cat', categorical_transformer, categorical_cols)
])


# Model pipeline
clf_model = Pipeline([
    ('prep', preprocessor_cls),
    (('model', LogisticRegression(max_iter=3000))
)
])

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X_cls, y_cls, test_size=0.2, random_state=42
)

# Train & Predict
clf_model.fit(X_train, y_train)
y_pred = clf_model.predict(X_test)


# Evaluation
cm = confusion_matrix(y_test, y_pred)
precision = precision_score(y_test, y_pred, pos_label='Yes')
recall = recall_score(y_test, y_pred, pos_label='Yes')

print("Confusion Matrix:\n", cm)
print("Precision:", round(precision, 2))
print("Recall:", round(recall, 2))
print("Recall matters because missing a follow-up patient can affect treatment outcomes.")



Confusion Matrix:
 [[21 14]
 [13  9]]
Precision: 0.39
Recall: 0.41
Recall matters because missing a follow-up patient can affect treatment outcomes.


C:\Users\User\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
## TASK 7 -> FINAL CONCLUSION
1. Pandas was used to clean up the hospital outpatient data by processing missing values, datatype errors, text standardization, and eliminating duplications.
2. It was indicated that the average billing is different among the departments, few visits are used to create high overall bills, and there are no high requirements of follow-up by most patients.
3. The number of bills was predicted using Linear Regression and the MAE represents the mean absolute difference between the real and the predicted amount of the bill.
4. The application of Logistic Regression model predicted the need to follow-up and precision was used to indicate the accuracy of positive prediction and recall indicated the ability to identify actual cases of follow-ups.
5. In this regard, recall is of particular importance since a follow-up patient not taken care of can adversely affect the results of the treatment.
6. The disadvantage of this analysis is that the initial data used is relatively small and this could have an influence on the generalization of the model to new patient data.